In [1]:
import tensorflow as tf
import networkx as nx
import numpy as np
import keras
from keras.layers import Dense
from keras.optimizers import Adam
import pandas as pd
import networkx as nx
import math
import random
import matplotlib.pyplot as plt
from collections import Counter
from queue import PriorityQueue
import csv
from datetime import datetime
import pytz
import os
import time
import copy


np.random.seed(42)
num_actions=8

class Env(object):
    def __init__(self):
        self.path = []
        self.path_no_relations = []
        self.path_relations = []
        self.coords_path = []
        self.curr_rewards = 0.0
        self.curr_prob = 1.0
        self.edge_rewards = {}
        self.edge_transition_probs = {}
        self.die = 0  # record how many times does the agent choose an invalid path
        self.last_visited = {}
        self.visited = {}
        self.onlyreward=0.0
        self.done = 0
        self.steps = 0
        self.path_length = 0.0

    def copy(self):
        o = Env.__new__(Env)
        o.path = list(self.path)
        o.path_relations = list(self.path_relations)
        o.path_no_relations = list(self.path_no_relations)
        o.onlyreward = self.onlyreward
        o.coords_path = list(self.coords_path)
        o.curr_rewards = self.curr_rewards
        o.curr_prob = self.curr_prob
        o.edge_rewards = {k: dict(v.items()) for k, v in self.edge_rewards.items()}
        o.edge_transition_probs = {k: dict(v.items()) for k, v in self.edge_transition_probs.items()}
        o.die = self.die  # record how many times does the agent choose an invalid path
        o.last_visited = {k: tuple(v) for k, v in self.last_visited.items()}
        o.visited = {k: set(v) for k, v in self.visited.items()}
        o.done = self.done
        o.steps = self.steps
        o.path_length = self.path_length
        return o

    def __lt__(self, other):
        # Define the comparison logic between instances of Env
        return self.curr_prob < other.curr_prob

    # def __lt__(self, other):
    #     # Define the comparison logic between instances of Env
    #     return self.curr_prob < other.curr_prob

    def interact(self, state, action, rollout, prob):
        done = 0  # Whether the episode has finished
        cur = state[0]
        dest = state[1]
        next_node,green,aqi,noise,length=get_next_state(G,cur,action)
        if(next_node==None):
            print('the action is invalid ,no next state from here')
        else:
            edge_reward = calculate_combined_reward(aqi, green, noise,length)
            self.path.append(str(action)+'->'+next_node) #keno use hocce buji nai
            self.path_no_relations.append(next_node) #keno use hocce buji nai
            self.coords_path.append((G.nodes[next_node]['latitude'],G.nodes[next_node]['longitude'])) #keno use hocce buji nai

            # add action taken to dictionary of state-action pairs
            self.last_visited[cur] = (action,next_node,len(self.path) - 1) #3rd param buji nai
            if cur in self.visited:
                self.visited[cur].add(action)
            else:
                self.visited[cur] = {action}

            self.die = 0
            if next_node == dest and not rollout:
                done = 1
                self.done = 1


        # save above reward data for each node visited for future rewards
        if cur in self.edge_rewards:
            if next_node not in self.edge_rewards[cur]:
                self.edge_rewards[cur][next_node] = (aqi,green,noise,length,edge_reward)
        else:
            self.edge_rewards[cur] = {next_node:(aqi,green,noise,length,edge_reward)}

        # save probabilities for each node taken
        if cur in self.edge_transition_probs:
            if next_node not in self.edge_transition_probs[cur]:
                self.edge_transition_probs[cur][next_node] = prob
        else:
            self.edge_transition_probs[cur] = {next_node: prob}

        # print('in interact methdo: ')
        # print('cur,nest: ',cur,next_node)
        # print('probs: ',self.edge_transition_probs[cur][next_node])
        next_state=(next_node,dest,self.die)
        self.curr_rewards += edge_reward
        self.curr_prob *= prob
        self.steps += 1
        self.onlyreward+=edge_reward
        self.path_length += G[cur][next_node]['e_l']
        # print('len: ',G[cur][next_node]['e_l'],cur,next_node,edge_reward)
        # exit(0)
        return edge_reward, next_state,G[cur][next_node]['e_l'], done

class PolicyNetwork(tf.keras.Model):
    def __init__(self, num_actions,learning_rate=0.0005, seed=None):
        super(PolicyNetwork, self).__init__()
        self.initializer = keras.initializers.GlorotUniform(seed=seed)
        self.dense1 =Dense(512, activation='relu',kernel_initializer=self.initializer)
        self.dense2 =Dense(1024, activation='relu',kernel_initializer=self.initializer)
        self.output_layer =Dense(num_actions, activation='softmax',kernel_initializer=self.initializer)
        self.optimizer = Adam(learning_rate=learning_rate)

    def call(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        return self.output_layer(x)

    def update(self, state, action):
        with tf.GradientTape() as tape:
            action_prob = self(state)
            #print('action type in model: ',type(action),action)
            action_mask = tf.one_hot(action, depth=num_actions)
            picked_action_prob = tf.reduce_sum(action_prob * action_mask, axis=1)
            loss = tf.reduce_sum(-tf.math.log(picked_action_prob ))+sum(tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES))
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    def reinforce_update(self, states, actions, rewards, gamma=0.4):
        # Calculate the discounted returns for each time step
        G = []
        discounted_reward = 0
        for r in rewards[::-1]:  # Reverse the rewards and iterate
            discounted_reward = r + gamma * discounted_reward  # Apply discount factor
            G.insert(0, discounted_reward)  # Insert at the beginning to maintain order

        with tf.GradientTape() as tape:
            loss = 0
            for state, action, G_t in zip(states, actions, G):
                state = tf.expand_dims(tf.convert_to_tensor(state, dtype=tf.float32), 0)  # Convert to tensor and add batch dimension
                action_probs = self(state)[0]  # Get action probabilities from the policy network
                # print('action_probs: ',action_probs)
                action = tf.constant([action])
                p_a = tf.gather(action_probs[0], action)
                # print('g_t: ',G_t   )
                # print('p_a: ',p_a)  
                l=-tf.math.log(p_a)
                # print('l: ',l)
                loss += (l*G_t)  # Calculate the loss for this step

        gradients = tape.gradient(loss, self.trainable_variables)  # Compute gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))  # Apply gradients to update the model weights

        return loss.numpy()

    def predict(self, state):
        action_prob = self(state, training=False)
        return action_prob.numpy()


def calculate_initial_compass_bearing(pointA, pointB):
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing



def id_angle(angle):
    if angle <= 22.5 or angle > 337.5:
        return 0
    elif angle > 22.5 and angle <= 67.5:
        return 1
    elif angle > 67.5 and angle <= 112.5:
        return 2
    elif angle > 112.5 and angle <= 157.5:
        return 3
    elif angle > 157.5 and angle <= 202.5:
        return 4
    elif angle > 202.5 and angle <= 247.5:
        return 5
    elif angle > 247.5 and angle <= 292.5:
        return 6
    elif angle > 292.5 and angle <= 337.5:
        return 7



def calculate_combined_reward(aqi, green, noise,p,p1):
    a=b=c=0
    if aqi >=7:
        a = 100
    elif aqi >=4:
        a = 60
    elif aqi >=0:
        a = 30
    if green >=7:
        b = 100
    elif green >=4:
        b = 60
    elif green >=0:
        b = 30
    if noise >=7:
        c = 100
    elif noise >=4:
        c = 60
    elif noise >=0:
        c = 30
    x=math.log(a)*aqi
    y=math.log(b)*green
    z=math.log(c)*noise
    w=.000000001
    # print('1st: ',x,y,z,x+y+z) 
    # print('plen: ',plen,norm)
    # print('norm plean: ',plen/norm)
    # print('w: ',w*plen)
    fl=((y+x+z)*20)+w*(p/p1)
    # print('fl: ',fl)
    return fl


def get_next_state(graph,node,action_chosen):
    next_chosen_node=None
    next_nn = list(graph.neighbors(node))
    actions=[]
    for n in  next_nn:
        action=id_angle(calculate_initial_compass_bearing((G.nodes[node]['latitude'],G.nodes[node]['longitude']),(G.nodes[n]['latitude'],G.nodes[n]['longitude'])))
        if(action==action_chosen):
            actions.append(n)
    # print('len of same action: ',len(actions))
    next_chosen_node=random.choice(actions)
    green=graph[node][next_chosen_node]['e_g']
    aqi=graph[node][next_chosen_node]['aqi']
    len=graph[node][next_chosen_node]['e_l']
    noise=graph[node][next_chosen_node]['e_n']
    return next_chosen_node,green,aqi,noise,len

def state_embedding(state_idx,embed):
    curr=embed[str(state_idx[0])]
    targ=embed[str(state_idx[1])]
    state=np.expand_dims(np.concatenate((np.asarray(curr), np.asarray(targ) - np.asarray(curr))), axis=0)
    return state

def _normalize_probs(p):
    if sum(p)>0:
#         print('sum: ',sum(p))
        p=[i+.000000001 for i in p]
        
        p/=sum(p)
    else:
        p = np.full_like(p, 1.0 / len(p))
    # Check if the sum is close to zero
   
    return p

G = pd.read_pickle("../input/normgraph/reversed_normalized_graph.gpickle")



def create_modified_graph(original_graph):
    modified_graph = original_graph.copy()
    for edge in modified_graph.edges(data=True):
        # Modify green view index attribute
        edge[2]['e_g'] = 10 - edge[2]['e_g'] + 1
    return modified_graph

# Create the modified graph
modified_G = create_modified_graph(G)

training_file='../input/normgraph/train_data_2000_10_150.txt'
# training_file='train_data_2000_10_150.txt'
test_file='../input/normgraph2/test_data_2000_10_30.txt'
# test_file='test_data_2000_10_30.txt'
emd_file='../input/normgraph/node2vec_graph_num.emd'
# emd_file='node2vec_graph_num.emd'

with open(emd_file,'r') as file:
    data=file.readlines()
node_embeddings={}
for line in data:
    node_embeddings[(line.split()[0])] = [float(x) for x in line.split()[1:]]
policy_network = PolicyNetwork(num_actions,seed=42)
failure=0
r_cn=0
g_cn=0

def choose_action_with_epsilon_greedy(action_probs, available_actions, epsilon=.1):
    global r_cn, g_cn
    if np.random.rand() < epsilon:  # With probability epsilon, choose a random action
        r_cn += 1
        return np.random.choice(available_actions)
    else:
        g_cn += 1
        # Set the probabilities of unavailable actions to 0
        adjusted_probs = np.zeros_like(action_probs)
        for action_index in available_actions:
            if action_index < len(action_probs):
                adjusted_probs[action_index] = action_probs[action_index]

        # Normalize the probabilities of available actions to sum to 1
        if adjusted_probs.sum() > 0:
            adjusted_probs /= adjusted_probs.sum()
        else:
            # If all adjusted_probs are 0 (unlikely but possible), treat it as a random choice among available actions
            return np.random.choice(available_actions)

        # Use np.random.choice to select an action based on the adjusted probabilities
        return np.random.choice(len(action_probs), p=adjusted_probs)


def lat_long_to_cartesian(lat, lon, R=6371):  # R is the Earth's radius in kilometers
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)

    x = R * math.cos(lat_rad) * math.cos(lon_rad)
    y = R * math.cos(lat_rad) * math.sin(lon_rad)
    return x, y

def euclidean_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2)



def run_pretrain(G,data):
    episode=0
    for row in data:
        episode+=1
        node=row.split(',')
        src=int(node[0])
        dest=int(node[1][:-1])
        # print('source ,dest: for episode ',src,dest,episode)
        dijkstra_path = nx.dijkstra_path(G,src,dest,weight='e_l')

        with(tf.device('/device:GPU:0')):
            for i in range(len(dijkstra_path)-1):
                state = state_embedding((dijkstra_path[i],dest), node_embeddings)
                action=id_angle(calculate_initial_compass_bearing((G.nodes[dijkstra_path[i]]['latitude'], G.nodes[dijkstra_path[i]]['longitude']),(G.nodes[dijkstra_path[i+1]]['latitude'], G.nodes[dijkstra_path[i+1]]['longitude'])))
                policy_network.update(state,action)

        if(episode%10==0):
            print('epi done: ',episode)


def run_training_episodes(G,data):
    global failure
    path_model=[]
    episode=0
    print("hi",len(data))

    for row in data:
        episode+=1
        node=row.split(',')
        src=int(node[0])
        dest=int(node[1][:-1])
        source_node=src
        destination_node=dest
        # print(type(source_node),type(destination_node))
        # print('source ,dest: for episode ',source_node,destination_node,episode)
        shortest_path_length = nx.shortest_path_length(G, source_node, destination_node, weight='e_l')

        best_path=None
        ga=None
        gn=None
        gg=None

        # print('dijkstra path: ',np.max(aqi_dijkstra),np.min(green_dijkstra),np.max(noise_dijkstra))
        # s_path = nx.shortest_path(G,source_node,destination_node,weight='e_l')
        num_of_paths=0
        for j in range(1500):
            source_node=src
            destination_node=dest
            state_idx=[source_node,destination_node]
            visited = [False] * 6000
            visited[source_node] = True
            done=0
            traversed_nodes=[source_node]
            p_len=0
            for _ in range(6000):
                cur_state=state_embedding(state_idx,node_embeddings)

                action_probs = policy_network.predict(cur_state)

                neighbor_nodes = list(G.neighbors(state_idx[0]))
                available_actions=[]
                available_neighbour_nodes=[]
                for n in  neighbor_nodes:
                    action=id_angle(calculate_initial_compass_bearing((G.nodes[state_idx[0]]['latitude'],G.nodes[state_idx[0]]['longitude']),(G.nodes[n]['latitude'],G.nodes[n]['longitude'])))
                    # don't choose an action that has been chosen before in the same state
                    if visited[n] == False:
                        available_actions.append(action)
                        available_neighbour_nodes.append(n)

                if len(available_actions)==0:
                    # print('no available actions')
                    break


                action_probs_avail = []
                choices_idx = []
                for c in available_actions:
                    choices_idx.append(c)
                    action_probs_avail.append(action_probs[0][c])
                norm_action_probs = _normalize_probs(action_probs_avail)
                
                if np.random.random() < 0.1:
                    # With 10% probability, choose a random action
                    action_chosen = np.random.choice(available_actions)
                else:
                    # Choose an action based on the provided probability distribution
                    action_chosen = int(np.random.choice(available_actions, p=norm_action_probs))

                ind=choices_idx.index(action_chosen)
                chosen_neighbor_node = available_neighbour_nodes[ind]
                traversed_nodes.append(chosen_neighbor_node)

                visited[chosen_neighbor_node] = True
                p_len+=G[state_idx[0]][chosen_neighbor_node]['e_l']

                eudis=euclidean_distance(lat_long_to_cartesian(G.nodes[dest]['latitude'], G.nodes[dest]['longitude']),lat_long_to_cartesian(G.nodes[chosen_neighbor_node]['latitude'], G.nodes[chosen_neighbor_node]['longitude']))
                
                if(p_len+eudis>2*shortest_path_length):
                    # print('no path within twice the shortest path length')
                    break
                
                if chosen_neighbor_node == destination_node:
                    done=1
                    break

                state_idx=[chosen_neighbor_node,destination_node]

            if(done==1):
                # print(f'Successfully reach destination in retrain number {j+1} in episode {episode} need step: {i+1}')
                num_of_paths+=1

                states = []
                actions = []
                rewards = []
                rl_aqi=[]
                rl_noise=[]
                rl_green=[]

                for i in range(len(traversed_nodes) - 1):
                    state_idx=[traversed_nodes[i], destination_node]
        #             print('whew wrong: ',state_idx[0],state_idx[1])
                    state = state_embedding(state_idx, node_embeddings)

                    action = id_angle(calculate_initial_compass_bearing(
                        (G.nodes[traversed_nodes[i]]['latitude'], G.nodes[traversed_nodes[i]]['longitude']),
                        (G.nodes[traversed_nodes[i + 1]]['latitude'], G.nodes[traversed_nodes[i + 1]]['longitude'])
                    ))
                    green=G[traversed_nodes[i]][traversed_nodes[i + 1]]['e_g']
                    aqi=G[traversed_nodes[i]][traversed_nodes[i + 1]]['aqi']
                    length=G[traversed_nodes[i]][traversed_nodes[i + 1]]['e_l']
                    noise=G[traversed_nodes[i]][traversed_nodes[i + 1]]['e_n']
                    rl_aqi.append(aqi)
                    rl_noise.append(noise)
                    rl_green.append(green)
                    reward = calculate_combined_reward(aqi, green, noise,shortest_path_length,p_len)
                    states.append(state)
                    actions.append(action)
                    rewards.append(reward)


                policy_network.reinforce_update(states, actions, rewards)

                # print('success in episode: ',j)
                # print('our path: ',np.max(rl_aqi),np.min(rl_green),np.max(rl_noise))
                if ga==None:
                    ga=np.min(rl_aqi)
                else:
                    if(np.min(rl_aqi)>ga):
                        ga=np.min(rl_aqi)
                        best_path=traversed_nodes
                if gn==None:
                    gn=np.min(rl_noise)
                else:
                    if(np.min(rl_noise)>gn):
                        gn=np.min(rl_noise)
                        best_path=traversed_nodes
                if gg==None:
                    gg=np.min(rl_green)
                else:
                    if(np.min(rl_green)>gg):
                        gg=np.min(rl_green)
                        best_path=traversed_nodes
            else:
                # print('ist sn: ',s_n,cnt)
                pass

        if(best_path!=None):
            path_model.append(best_path)
        else:
            failure+=1
        print('num of paths: epi',num_of_paths,episode)

    with open('exp12_train.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(path_model)

        
with open(training_file,'r') as file:
        train_data=file.readlines()
# run_pretrain(G,train_data)

run_training_episodes(G,train_data)
print('fail:',failure,r_cn,g_cn)

import heapq
import time



def beam_search(graph, start, end, k=5, step_limit=6000):
    start_time = time.perf_counter_ns()  # Record the start time

    paths = [(1, [start], {start})]  # (cost, path, visited)
    final_paths = []
    steps = 0

    while paths and len(final_paths) < 5 and steps < step_limit:
        new_paths = []
        for cost, path, visited in paths:
            current = path[-1]
            if current == end:
                final_paths.append((cost, path))
                if len(final_paths) == 5:
                    break
                continue

            if current in graph:
                
                cur_state=state_embedding((current,end),node_embeddings)

                action_probs = policy_network.predict(cur_state)

                neighbor_nodes = list(G.neighbors(current))
                available_actions=[]
                available_neighbour_nodes=[]
                available_probs=[]
                for n in  neighbor_nodes:
                    action=id_angle(calculate_initial_compass_bearing((G.nodes[current]['latitude'],G.nodes[current]['longitude']),(G.nodes[n]['latitude'],G.nodes[n]['longitude'])))
                    # don't choose an action that has been chosen before in the same state
  
                    available_actions.append(action)
                    available_neighbour_nodes.append(n)
                    available_probs.append(action_probs[0][action])

                if len(available_actions)==0:
#                     print('no available actions')
                    break
                
                available_probs = _normalize_probs(available_probs)

                for next_node in available_neighbour_nodes:
                    if next_node not in visited:
                        new_visited = visited.copy()
                        new_visited.add(next_node)
                        new_path = path + [next_node]
                        new_cost = cost+ (-math.log(available_probs[available_neighbour_nodes.index(next_node)]))
                        new_paths.append((new_cost, new_path, new_visited))

        paths = heapq.nlargest(k, new_paths, key=lambda x: x[0])
        steps += 1

    end_time = time.perf_counter_ns()  # Record the end time

    if final_paths:
        best_path = max(final_paths, key=lambda x: x[0])[1]
    else:
        best_path = None

    execution_time = (end_time - start_time)/1000000  # Calculate the execution time

    return best_path, execution_time


with open(test_file, 'r') as file:
    test_data = file.readlines()

episode=0
time_taken_arr=[]
sucess_test_Rate=0
test_paths=[]
for row in test_data:
    episode+=1
    node=row.split(',')
    src=int(node[0])
    dest=int(node[1][:-1])

    path, time_taken = beam_search(G, src,dest)
    print("Path found:", path)
    print("Time taken (seconds):", time_taken)
    if path is not None:
        sucess_test_Rate+=1
        test_paths.append(path)
        time_taken_arr.append(time_taken)

print("Average time taken (seconds):", sum(time_taken_arr) / len(time_taken_arr))
print("Success rate:", sucess_test_Rate,30-sucess_test_Rate,sucess_test_Rate / len(test_data))

with open('exp12_test.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(test_paths)

2024-06-13 12:59:52.517291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 12:59:52.517449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 12:59:52.662969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


hi 150


KeyboardInterrupt: 